In [ ]:
import numpy as np
import time
import cv2
import os
import zlib
from sdlarch_rl import make
import pygame
from IPython.display import Audio
from stable_baselines3 import PPO
from stable_baselines3.common.atari_wrappers import WarpFrame, MaxAndSkipEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from sdlarch_rl.utils.utils import get_latest_model, TrainAndLoggingCallback
from sdlarch_rl.utils.discretizer import MainDiscretizer
from stable_baselines3.common.vec_env import VecFrameStack
from stable_baselines3.common.callbacks import CallbackList, EvalCallback
from pathlib import Path
from gymnasium.wrappers import TimeLimit

NUM_ENV = 4
SAVE_DIR="./model-gt3"
TENSORBOARD="./tensorboard-gt3"
TOTAL_TIMESTEP_NUMB = 500_000_000
CHECK_FREQ_NUMB = 5_000
SAVE_FREQ = CHECK_FREQ_NUMB
MAX_STEPS= 2_000

SAVE_DIR = Path(SAVE_DIR)
combos = [
    [],
    ["LEFT"],
    ["RIGHT"],
    ["Y"], # break
    ["B"], # ac
    ["LEFT", "B"],
    ["RIGHT", "B"],
]


def make_env():
    def _init():
        env = make(
            "GranTurismo3-Ps2", 
            render_mode="human"
        )

        env = MainDiscretizer(
            env,
            combos,
        )

        env = WarpFrame(env, width=96, height=96)
        # env = WarpFrame(env)
        env = MaxAndSkipEnv(env, skip=4)
        env = TimeLimit(env, MAX_STEPS)

        return env
    return _init

    
# env = make_vec_env(make_env(), n_envs=NUM_ENV)
env = make_vec_env(make_env(), n_envs=NUM_ENV, vec_env_cls=SubprocVecEnv)
env = VecFrameStack(env, 4, channels_order='last')

latest_model_path = get_latest_model(SAVE_DIR)

if latest_model_path:
    print(f"Loading existent model: {latest_model_path}")
    model = PPO.load(
    # model = RecurrentPPO.load(
        str(latest_model_path), 
        env=env, 
        verbose=0, 
        tensorboard_log=TENSORBOARD, 
    )
    
else:
    print("None finded, starting from zero.")
    model = PPO('CnnPolicy', 
    # model = RecurrentPPO('CnnLstmPolicy',
        env, 
        verbose=0, 
        # policy_kwargs=policy_kwargs, 
        tensorboard_log=TENSORBOARD, 
    )

# eval_callback = EvalCallback(
#     eval_env, 
#     best_model_save_path="./logs/best_model",
#     log_path="./logs/results", 
#     eval_freq=5_000,
#     n_eval_episodes=6,
#     deterministic=True
# )

checkpoint_callback=TrainAndLoggingCallback(check_freq=CHECK_FREQ_NUMB, save_path=SAVE_DIR, save_freq=SAVE_FREQ, model=model)
#callback = CallbackList([checkpoint_callback, eval_callback])
callback = CallbackList([checkpoint_callback])

model.learn(total_timesteps=TOTAL_TIMESTEP_NUMB, reset_num_timesteps=False, callback=callback)
model.save("final_gt3")

env.close()

D:\Python311\Lib\site-packages\pygame\pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists


Loading existent model: model-gt3\best_model_294000
